In [24]:
from pynq import Overlay
from pynq.lib import AxiGPIO
from pynq import Interrupt
from time import sleep
import asyncio
ol = Overlay("block_design_top.bit")

input_to_lock = AxiGPIO(ol.ip_dict['axi_gpio_0']).channel2
output_from_lock = AxiGPIO(ol.ip_dict['axi_gpio_0']).channel1

input_to_lock.setdirection("out")
input_to_lock.setlength(8)
output_from_lock.setdirection("in")
output_from_lock.setlength(8)

correct_bits_count_line = input_to_lock[0:3]
error_line = input_to_lock[4]
locked_line = input_to_lock[5]
new_seq_ack_line = input_to_lock[6]
sequence_correct = input_to_lock[7]

new_seq_ready = Interrupt('top_0/new_seq_ready')
lock = Interrupt('top_0/lock')
rst = Interrupt('input_interface_0/rst')

current_sequence = 0
secret_sequence = 0
current_correctly_set_bits = 0 # link this to the respective gpio lines
waiting_for_new_secret = True

def init():
    global waiting_for_new_secret
    global secret_sequence
    global current_sequence
    global current_correctly_set_bits
    secret_sequence = 0
    current_sequence = 0
    current_correctly_set_bits = 0
    correct_bits_count_line.write(current_correctly_set_bits)
    waiting_for_new_secret = True
    locked_line.on()
    new_seq_ack_line.off()
    ol.axi_intc_0.register_map.IER = 0xF
    ol.axi_intc_0.register_map.MER = 3
    return

init()

In [25]:
def get_bit(num, index):
    return (num >> index) & 1

def pulse(line):
    line.on() # TODO: see if there's a better way to do this
    sleep(0.5)
    line.off()

async def wait_for_new_sequence():
    global current_sequence
    while True:
        await new_seq_ready.wait()
        current_sequence = output_from_lock.read()
        pulse(new_seq_ack_line)
        handle_new_sequence()

def handle_new_sequence():
    global current_correctly_set_bits
    global secret_sequence
    if not waiting_for_new_secret:
        check_sequence(current_sequence)
        print('entered: {}   bits correct: {}'.format(bin(current_sequence), current_correctly_set_bits))
    else:
        current_correctly_set_bits = current_correctly_set_bits + 1
        if current_correctly_set_bits == 8:
            current_correctly_set_bits = 0
            secret_sequence = current_sequence
            waiting_for_new_secret = False
            locked_line.off()
            print('set new secret sequence to: {}'.format(bin(secret_sequence)))
        else:
            print('entered: {}   next editing bit: {}'.format(bin(current_sequence), current_correctly_set_bits))
        correct_bits_count_line.write(current_correctly_set_bits)
        
async def set_new_sequence():
    while True:
        await lock.wait()
        init()
        print('setting new secret')

async def reset():
    while True:
        await rst.wait()
        init()
        print('reset issued, awaiting new secret')
    
def check_sequence(sequence):
    next_bit = current_correctly_set_bits + 1
    if get_bit(sequence, next_bit) == get_bit(secret_sequence, next_bit):
        current_correctly_set_bits += 1
        if current_correctly_set_bits == 8:
            signal_unlock()
    else:
        raise_error()
    correct_bits_count_line.write(current_correctly_set_bits)

def raise_error():
    error_line.on()
    sleep(1)
    error_line.off()

def signal_unlock():
    sequence_correct.on()
    sleep(1)
    sequence_correct.off()

async def main():
    task1 = asyncio.create_task(wait_for_new_sequence())
    task2 = asyncio.create_task(set_new_sequence())
    task3 = asyncio.create_task(reset())
    await task1
    await task2
    await task3

In [38]:
print(ol.axi_intc_0.register_map.ISR)
print(ol.axi_intc_0.register_map.IPR)
print(ol.axi_intc_0.register_map.IER)
ol.axi_intc_0.register_map.MER

0x1
0x1
0x7


Register(ME=1, HIE=1)

In [36]:
ol.axi_intc_0.register_map.IAR = 0x7

In [5]:
asyncio.run(main())

setting new secret
reset issued, awaiting new secret


KeyboardInterrupt: 

In [39]:
bin(output_from_lock.read())

'0b1'

In [88]:
signal_unlock()

In [181]:
ol.axi_intc_0.register_map.IPR

Register(INT=0)